In [1]:
# Add your imports here
import pandas as pd
import numpy as np
import scipy as sp
import findspark
import os
findspark.init(os.environ['SPARK_HOME'])
from pyspark.sql import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt

%matplotlib inline

spark = SparkSession.builder.getOrCreate()


In [2]:
food_facts = spark.read.csv('../data/raw/en.openfoodfacts.org.products.csv', header=True, sep="\t")
food_facts.registerTempTable('food_facts')

In [3]:
food_facts.printSchema()

root
 |-- code: string (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: string (nullable = true)
 |-- created_datetime: string (nullable = true)
 |-- last_modified_t: string (nullable = true)
 |-- last_modified_datetime: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- categories_en: string (nullable = true)
 |-- origins: string (nullable = true)
 |-- origins_tags: string (nullable = true)
 |-- manufacturing_places: string (nullable = true)
 |-- manufacturing_places_tags: string (nullable = true)
 |-- labels: string (nullable = true)
 |-- labels_tags: string (nullable 

# First exploration of the data

In [4]:
number_of_products = food_facts.count()
print("We have {} products in our dataframe.".format(number_of_products))

We have 699124 products in our dataframe.


In [5]:
print("Number of distinct city tags without france: {}".format(
    food_facts.select('cities_tags').filter('cities_tags NOT LIKE "%france%"')
    .distinct().count())
)
print("Number of distinct city tags : {}".format(
    food_facts.select('cities_tags').distinct().count())
)

Number of distinct city tags without france: 3
Number of distinct city tags : 3332


In [6]:
print("Number of distinct purchase places without france in the name: {}".format(
    food_facts.select('purchase_places').filter('LOWER(purchase_places) NOT LIKE "%france%"')
    .distinct().count()))
print("Number of distinct purchase places : {}".format(
    food_facts.select('purchase_places').distinct().count()))

Number of distinct purchase places without france in the name: 3480
Number of distinct purchase places : 7822


In [7]:
print("Number of manufacturing places without france in the name: {}".format(
    food_facts.select('manufacturing_places_tags')
              .filter('LOWER(manufacturing_places_tags) NOT LIKE "%france%"')
              .distinct().count())
)
print("Number of manufacturing places : {}".format(
    food_facts.select('manufacturing_places_tags').distinct().count())
)

Number of manufacturing places without france in the name: 7673
Number of manufacturing places : 12080


It seems more interesting to make an analysis focused on the French situation rather than the situation in the US.

In [8]:
food_facts.select('generic_name').filter('generic_name IS NOT NULL') \
    .distinct().toPandas().head()

,generic_name
0,Banana & Berries - Banane et baies
1,Boisson au jus de cranberry
2,Bebida de almendras UHT con calcio y vitaminas
3,Cereal de maiz inflado endulzado
4,Base en pâte brisée avec du bœuf d'origine bri...


We'll probably have to filter out the names that are not present in French.

In [9]:
nutrition_grades = food_facts.select(
    'nutrition_grade_fr', 'nutrition_grade_uk', 'code', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g'
    ).filter('nutrition_grade_fr IS NOT NULL OR nutrition_grade_uk IS NOT NULL OR `nutrition-score-uk_100g` IS NOT NULL OR `nutrition-score-fr_100g` IS NOT NULL').toPandas()
print("""Total number of products for which we have nutrition informations: 
{}"""
      .format(nutrition_grades.count())
)
print("Proportion of products for which we have nutrition informations: {ratio:.3%}"
      .format(ratio=(nutrition_grades['code'].count()/number_of_products))
)
print("Description of the table: ")
nutrition_grades.describe()

Total number of products for which we have nutrition informations: 
nutrition_grade_fr         141680
nutrition_grade_uk              0
code                       141680
nutrition-score-fr_100g    141680
nutrition-score-uk_100g    141680
dtype: int64
Proportion of products for which we have nutrition informations: 20.265%
Description of the table: 


,nutrition_grade_fr,nutrition_grade_uk,code,nutrition-score-fr_100g,nutrition-score-uk_100g
count,141680,0.0,141680,141680,141680
unique,5,0.0,141676,56,56
top,d,NaN,3263851538918,0,0
freq,41275,NaN,2,7380,10207


We see that `nutrition_grade_uk` is useless.

## Data cleaning

## Food

In [10]:
# select only the columns we plan to use
food_dataframe = food_facts.select('generic_name',
                                   'code',
                                   'nutrition_grade_fr',
                                   'nutrition-score-fr_100g',
                                   'nutrition-score-uk_100g',
                                   'serving_size',
                                   'energy_100g',
                                   'energy-from-fat_100g', 
                                   'trans-fat_100g',
                                   'fat_100g',
                                   'saturated-fat_100g',
                                   'monounsaturated-fat_100g',
                                   'polyunsaturated-fat_100g',
                                   'cholesterol_100g',
                                   'proteins_100g',
                                   'carbohydrates_100g', 
                                   'sugars_100g', 
                                   'fiber_100g',
                                   'fruits-vegetables-nuts_100g',
                                   'fruits-vegetables-nuts-estimate_100g',
                                   'glycemic-index_100g',
                                   'cities',
                                   'cities_tags',
                                   'purchase_places',
                                   'stores',
                                   'countries',
                                   'countries_tags')
# move to pandas for the rest of the analysis
food_dataframe = food_dataframe.toPandas()

We first remove all products that are not sold in France.

In [11]:
food_dataframe = food_dataframe[np.logical_not(food_dataframe['countries_tags'].apply(
                                        lambda x: x is None or "france" not in x.lower())
               & food_dataframe['countries'].apply(lambda x: x is None or "france" not in x.lower())
               & food_dataframe['cities_tags'].apply(lambda x: x is None or "france" not in x.lower())
               & food_dataframe['cities'].apply(lambda x: x is None or "france" not in x.lower())
               & food_dataframe['purchase_places'].apply(lambda x: x is None or "france" not in x.lower())
               & food_dataframe['stores'].apply(lambda x: x is None or "france" not in x.lower()))]

Now, we want to check how many of our features have a significant amount of entries.

In [12]:
total_rows = len(food_dataframe)
for col in food_dataframe.columns:
    none_values = np.count_nonzero(food_dataframe[col].apply(lambda x: x is None))
    not_none_percentage = (total_rows - none_values) / total_rows
    print('Rows that are **not** None in {col}: {p:.3%}'.format(col=col, p=not_none_percentage))

Rows that are **not** None in generic_name: 12.453%
Rows that are **not** None in code: 100.000%
Rows that are **not** None in nutrition_grade_fr: 26.243%
Rows that are **not** None in nutrition-score-fr_100g: 26.243%
Rows that are **not** None in nutrition-score-uk_100g: 26.243%
Rows that are **not** None in serving_size: 11.005%
Rows that are **not** None in energy_100g: 86.242%
Rows that are **not** None in energy-from-fat_100g: 0.025%
Rows that are **not** None in trans-fat_100g: 0.308%
Rows that are **not** None in fat_100g: 85.178%
Rows that are **not** None in saturated-fat_100g: 85.903%
Rows that are **not** None in monounsaturated-fat_100g: 0.429%
Rows that are **not** None in polyunsaturated-fat_100g: 0.435%
Rows that are **not** None in cholesterol_100g: 0.302%
Rows that are **not** None in proteins_100g: 86.176%
Rows that are **not** None in carbohydrates_100g: 85.112%
Rows that are **not** None in sugars_100g: 85.929%
Rows that are **not** None in fiber_100g: 25.204%
Rows 

**Based on this, we will have to remove some rows:**

We remove the rows for which we have less than 10% of not None values.

- energy-from-fat_100g
- monounsaturated-fat_100g
- polyunsaturated-fat_100g
- cholesterol_100g
- trans-fat_100g
- fruits-vegetables-nuts_100g
- fruits-vegetables-nuts-estimate_100g
- glycemic-index_100g
- cities
- generic_name

Since we have the code of the product, we drop the generic_name attribute too.

We also need to drop all rows for which we have no information whatsoever (nutrition_grade, fat, saturated fat, sugars, proteins, fiber, carbohydrates, energy).

Finally, we need to drop all rows for which we have no information on the city where it is sold.

In [13]:
# drop the columns with too few values
food_dataframe = food_dataframe.drop(columns=['energy-from-fat_100g', 
                             'monounsaturated-fat_100g', 
                             'polyunsaturated-fat_100g',
                             'cholesterol_100g',
                             'trans-fat_100g',
                             'fruits-vegetables-nuts_100g',
                             'fruits-vegetables-nuts-estimate_100g',
                             'glycemic-index_100g',
                             'glycemic-index_100g',
                             'cities',
                             'generic_name'])

# drop the rows with too little nutritional information
food_dataframe = food_dataframe[np.logical_not(food_dataframe['nutrition_grade_fr'].apply(lambda x: x is None)
                                               # we do not check the values of nutrition-score-fr_100g or
                                               # nutrition-score-uk_100g, because we know that they exists for exactly
                                               # the same rows as nutrition_grade_fr
                                               & food_dataframe['energy_100g'].apply(lambda x: x is None) 
                                               & food_dataframe['fat_100g'].apply(lambda x: x is None) 
                                               & food_dataframe['saturated-fat_100g'].apply(lambda x: x is None) 
                                               & food_dataframe['carbohydrates_100g'].apply(lambda x: x is None) 
                                               & food_dataframe['sugars_100g'].apply(lambda x: x is None) 
                                               & food_dataframe['fiber_100g'].apply(lambda x: x is None) 
                                               & food_dataframe['proteins_100g'].apply(lambda x: x is None)
                                              )
                               ]

# drop the rows with no purchase place
food_dataframe = food_dataframe[food_dataframe['purchase_places'].apply(lambda x: x is not None)]

In [14]:
food_dataframe.head()

,code,nutrition_grade_fr,nutrition-score-fr_100g,nutrition-score-uk_100g,serving_size,energy_100g,fat_100g,saturated-fat_100g,proteins_100g,carbohydrates_100g,sugars_100g,fiber_100g,cities_tags,purchase_places,stores,countries,countries_tags
108,0000000020114,None,None,None,16 ml,0,0,0,0,0,0,None,None,France,None,en:france,en:france
254,0000000274722,b,0,0,285 g,450,2.2,0.9,6.8,15.3,0.5,0.5,None,France,Comme J'aime,France,en:france
255,0000000274739,b,1,1,300 g,455,4.2,2.1,4.4,12.5,1.4,1.8,None,France,None,France,en:france
292,0000007020254,e,12,4,33 cl,213,0,0,0,14,0,0,None,"France,États-Unis",None,"France,États-Unis","en:france,en:united-states"
302,0000009125124,b,1,1,40 g + 100 ml d'eau,1577,0.5,0.1,1.5,93,66,None,None,France,None,France,en:france


In [15]:
total_rows = len(food_dataframe)
for col in list(food_dataframe.columns):
    none_values = np.count_nonzero(food_dataframe[col].apply(lambda x: x is None))
    not_none_percentage = (total_rows - none_values) / total_rows
    print('Rows that are **not** None in {col}: {p:.3%}'.format(col=col, p=not_none_percentage))

Rows that are **not** None in code: 100.000%
Rows that are **not** None in nutrition_grade_fr: 89.747%
Rows that are **not** None in nutrition-score-fr_100g: 89.747%
Rows that are **not** None in nutrition-score-uk_100g: 89.747%
Rows that are **not** None in serving_size: 49.843%
Rows that are **not** None in energy_100g: 99.508%
Rows that are **not** None in fat_100g: 99.086%
Rows that are **not** None in saturated-fat_100g: 98.003%
Rows that are **not** None in proteins_100g: 99.243%
Rows that are **not** None in carbohydrates_100g: 98.726%
Rows that are **not** None in sugars_100g: 98.081%
Rows that are **not** None in fiber_100g: 54.563%
Rows that are **not** None in cities_tags: 30.393%
Rows that are **not** None in purchase_places: 100.000%
Rows that are **not** None in stores: 75.573%
Rows that are **not** None in countries: 99.980%
Rows that are **not** None in countries_tags: 99.980%


We still have rows for which the city tag is none and the purchase place is very vague, like France or the US. We need to take care of those. We will do it by creating a matching between the Open Food Facts dataset and a dataset including all cities of France.

## Cities

In [16]:
# The dataframe: 
# https://www.insee.fr/fr/statistiques/3126432
revenue_df = pd.read_excel("../data/raw/base-cc-filosofi-2014.xls", skiprows=[0,1,2,3])

# Drop rows that are not data and reset index
revenue_df.drop(0, inplace=True)
revenue_df.reset_index(inplace=True, drop=True)


# Drop useless columns
revenue_df = revenue_df.drop(columns=['Nombre de ménages fiscaux','Code géographique',
                                      'dont part des salaires, traitements ou chômage (%)',
                                      "dont part des revenus d'activités non salariées (%)",
                                      'dont part des prestations familiales (%)',
                                      'dont part des minima sociaux (%)',
                                      'dont part des prestations logement (%)',
                                      '1er décile du niveau de vie (€)',
                                      '9e décile du niveau de vie (€)',
                                      'Rapport inter-décile 9e décile/1er decile',
                                      'Part des ménages fiscaux imposés (%)'
                                     ]
                            )

# Translate the remaining columns to English
revenue_df = revenue_df.rename(columns={"ANNEE" : "Year",
                                        'Nombre de personnes dans les ménages fiscaux': "Household inhabitants", 
                                        'Médiane du niveau vie (€)': "Median revenue euros", 
                                        'Taux de pauvreté-Ensemble (%)' : "Total poverty rate (%)", 
                                        'Taux de pauvreté-moins de 30 ans (%)' : "Poverty rate (-30) (%)",
                                        'Taux de pauvreté-30 à 39 ans  (%)' : "Poverty rate (30-39) (%)",
                                        'Taux de pauvreté-40 à 49 ans (%)': "Poverty rate (40-49) (%)", 
                                        "Taux de pauvreté-50 à 59 ans (%)" :  "Poverty rate (50-59) (%)", 
                                        "Taux de pauvreté-60 à 74 ans (%)" :  "Poverty rate (60-74) (%)", 
                                        "Taux de pauvreté-75 ans ou plus (%)":  "Poverty rate (75+) (%)", 
                                        "Taux de pauvreté-propriétaires (%)" :  "Poverty rate (house owners) (%)", 
                                        "Taux de pauvreté-locataires (%)" :  "Poverty rate (tenants) (%)",
                                        "Part des revenus d'activité (%)" :  "Share of activity revenue (%)", 
                                        'Part des pensions, retraites et rentes (%)' :  "Share of retreat pension revenue (%)", 
                                        'Part des revenus du patrimoine et autres revenus (%)' :  "Share of heritage revenue and other (%)",  
                                        "Part de l'ensemble des prestations sociales (%)" :  "Share of social benefits revenue (%)", 
                                        'Part des impôts (%)' :  "Share of taxes (%)",
                                        'Libellé géographique' : "City name"
                                       }
                              )

In [17]:
revenue_df.head()

,City name,Household inhabitants,Median revenue euros,Total poverty rate (%),Poverty rate (-30) (%),Poverty rate (30-39) (%),Poverty rate (40-49) (%),Poverty rate (50-59) (%),Poverty rate (60-74) (%),Poverty rate (75+) (%),Poverty rate (house owners) (%),Poverty rate (tenants) (%),Share of activity revenue (%),Share of retreat pension revenue (%),Share of heritage revenue and other (%),Share of social benefits revenue (%),Share of taxes (%)
0,L'Abergement-Clémenciat,799.5,21576.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L'Abergement-de-Varey,235.5,21672.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ambérieu-en-Bugey,13660.5,19756.1,15.7534,19.4181,19.5204,19.1982,14.7159,NaN,NaN,5.40116,24.796,71.8,27.3,10.1,6.5,-15.7
3,Ambérieux-en-Dombes,1661.5,23204.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ambléon,102,22157.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print("Total number of rows:", len(revenue_df))

Total number of rows: 36621


In [19]:
print("Total number of cities: ", len(list(set(revenue_df["City name"]))))

Total number of cities:  34099


# Mapping cities - food products

In this section, we will try to find a mapping between a city as present in the OpenFoodFacts dataset, and a city as know by the INSEE.

In [20]:
# Helper to remove the accents
import unicodedata as ud

def remove_accents(input_str):
    nkfd_form = ud.normalize('NFKD', str(input_str))
    return u"".join([c for c in nkfd_form if not ud.combining(c)])

In [21]:
# First, let us remove the accents, lowercase everything, and replace apostrophes by carets
revenue_df['City name'] = revenue_df['City name'].apply(lambda x: remove_accents(x.lower().replace("'", "-"))) 


In [22]:
revenue_df.head()

,City name,Household inhabitants,Median revenue euros,Total poverty rate (%),Poverty rate (-30) (%),Poverty rate (30-39) (%),Poverty rate (40-49) (%),Poverty rate (50-59) (%),Poverty rate (60-74) (%),Poverty rate (75+) (%),Poverty rate (house owners) (%),Poverty rate (tenants) (%),Share of activity revenue (%),Share of retreat pension revenue (%),Share of heritage revenue and other (%),Share of social benefits revenue (%),Share of taxes (%)
0,l-abergement-clemenciat,799.5,21576.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,l-abergement-de-varey,235.5,21672.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,amberieu-en-bugey,13660.5,19756.1,15.7534,19.4181,19.5204,19.1982,14.7159,NaN,NaN,5.40116,24.796,71.8,27.3,10.1,6.5,-15.7
3,amberieux-en-dombes,1661.5,23204.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ambleon,102,22157.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# get the tags of the cities per food item:
# until now, the tags were a comma separated list of tags
# we first split them
cities_for_food = (food_dataframe['cities_tags'].str.split(',', expand=True)
                    # then we create one entry per couple (food item, city tag)
                    .stack()
                    # we remove the index, as we will need to keep the column containing the id of the food item
                    .reset_index()
                    # and we drop duplicates and unnecessary columns
                    .drop(columns='level_1').drop_duplicates())

In [24]:
# we move back to spark, because pandas is too weak
sdf_city_names = spark.createDataFrame(revenue_df['City name'].drop_duplicates().reset_index())
sdf_cities_for_food = spark.createDataFrame(cities_for_food)
sdf_cities_for_food.createTempView('cities_for_food')
sdf_city_names.createTempView('city_names')

In [25]:
# we join our two dataframes, explanation below
sdf_joined = spark.sql("""
SELECT city_names.index AS city_index, city_names.`City name` AS city_name, 
       aux.food_item_index, aux.city_tag_from_food_item
FROM city_names
JOIN (
    SELECT MAX(LENGTH(city_names.`City name`)) AS length_city_name, 
           cities_for_food.level_0 AS food_item_index, cities_for_food.`0` AS city_tag_from_food_item
    FROM cities_for_food
    JOIN city_names
    ON cities_for_food.`0` LIKE CONCAT(city_names.`City name`, '%')
    GROUP BY cities_for_food.level_0, cities_for_food.`0`
) AS aux
ON aux.city_tag_from_food_item LIKE CONCAT(city_names.`City name`, '%')
WHERE aux.length_city_name == LENGTH(city_names.`City name`)
""")

A city's tag seems to usually be 'city-s-name-department-name-country-name'.
A city's name is now 'city-s-name'.

Thus we can join on the condition: `city_s_tag LIKE city_s_name + '%'`.

Unfortunately, the city's name "Saint Alban", formatted as "saint-alban", will also match the tag "saint-alban-les-eaux", though these two cities maybe totally different and far away from one another.

Thus, we want to kep only the longest city's name matching the tag, hence:
```sql
SELECT MAX(LENGTH(city_names.`City name`)) AS length_city_name, 
           cities_for_food.level_0 AS food_item_index, cities_for_food.`0` AS city_tag_from_food_item
    FROM cities_for_food
    JOIN city_names
    ON cities_for_food.`0` LIKE CONCAT(city_names.`City name`, '%')
    GROUP BY cities_for_food.level_0, cities_for_food.`0`
```
returning the size of the longest matching city name.

We then have to re-do the same join and filter using the computed condition.

In [26]:
# we write our dataframe to parquet, to be able to reuse it without needing to recompute everything
sdf_joined.write.mode('overwrite').parquet("../data/interim/sdf_joined_city_names.parquet")

In [27]:
# now we still have to join our food items and our cities, using the mapping we managed to get above
# we switch back to pandas
pdf_joined = sdf_joined.toPandas()
pdf_joined.head()

,city_index,city_name,food_item_index,city_tag_from_food_item
0,4315,sainte-genevieve-sur-argence,473948,sainte-genevieve-sur-argence-aveyron-france
1,4315,sainte-genevieve-sur-argence,670213,sainte-genevieve-sur-argence-aveyron-france
2,4315,sainte-genevieve-sur-argence,439198,sainte-genevieve-sur-argence-aveyron-france
3,4315,sainte-genevieve-sur-argence,473957,sainte-genevieve-sur-argence-aveyron-france
4,4315,sainte-genevieve-sur-argence,473954,sainte-genevieve-sur-argence-aveyron-france


In [28]:
# we join our mapping with the food dataframe
food_df_for_join = pdf_joined.join(food_dataframe, on="food_item_index")
food_df_for_join.head()

,city_index,city_name,food_item_index,city_tag_from_food_item,code,nutrition_grade_fr,nutrition-score-fr_100g,nutrition-score-uk_100g,serving_size,energy_100g,...,saturated-fat_100g,proteins_100g,carbohydrates_100g,sugars_100g,fiber_100g,cities_tags,purchase_places,stores,countries,countries_tags
0,4315,sainte-genevieve-sur-argence,473948,sainte-genevieve-sur-argence-aveyron-france,3760087842005,c,3,3,None,837,...,5.4,13.5,1.5,1.5,2,"sainte-genevieve-sur-argence-aveyron-france,sa...","Flers-en-Escrebieux,France",Carrefour,en:france,en:france
1,4315,sainte-genevieve-sur-argence,670213,sainte-genevieve-sur-argence-aveyron-france,8480017474278,None,None,None,None,1029,...,None,17,0,None,None,sainte-genevieve-sur-argence-aveyron-france,"Rillieux-la-Pape,Rhône,Rhône-ALpes,France",Dia,France,en:france
2,4315,sainte-genevieve-sur-argence,439198,sainte-genevieve-sur-argence-aveyron-france,3596710349302,c,4,4,100 g,891,...,7.2,19,0.5,0.5,0,sainte-genevieve-sur-argence-aveyron-france,"Strasbourg,France","Simply,Simply Market",France,en:france
3,4315,sainte-genevieve-sur-argence,473957,sainte-genevieve-sur-argence-aveyron-france,3760087843231,b,2,2,180 g,883,...,5.9,19.1,0,0,None,"sainte-genevieve-sur-argence-aveyron-france,sa...",France,None,France,en:france
4,4315,sainte-genevieve-sur-argence,473954,sainte-genevieve-sur-argence-aveyron-france,3760087843019,d,11,11,None,1029,...,8.6,17,0,0,None,"sainte-genevieve-sur-argence-aveyron-france,sa...","France,Harnes",Simply Market,France,en:france


In [29]:
# we join again on the income dataframe
global_df = food_df_for_join.join(revenue_df, on="city_index")
# we can now drop the intermediary columns: city_name, and cities_tags
global_df.drop(columns=['cities_tags'], inplace=True)
global_df.head()

,city_index,city_name,food_item_index,city_tag_from_food_item,code,nutrition_grade_fr,nutrition-score-fr_100g,nutrition-score-uk_100g,serving_size,energy_100g,...,Poverty rate (50-59) (%),Poverty rate (60-74) (%),Poverty rate (75+) (%),Poverty rate (house owners) (%),Poverty rate (tenants) (%),Share of activity revenue (%),Share of retreat pension revenue (%),Share of heritage revenue and other (%),Share of social benefits revenue (%),Share of taxes (%)
0,4315,sainte-genevieve-sur-argence,473948,sainte-genevieve-sur-argence-aveyron-france,3760087842005,c,3,3,None,837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4315,sainte-genevieve-sur-argence,670213,sainte-genevieve-sur-argence-aveyron-france,8480017474278,None,None,None,None,1029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4315,sainte-genevieve-sur-argence,439198,sainte-genevieve-sur-argence-aveyron-france,3596710349302,c,4,4,100 g,891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4315,sainte-genevieve-sur-argence,473957,sainte-genevieve-sur-argence-aveyron-france,3760087843231,b,2,2,180 g,883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4315,sainte-genevieve-sur-argence,473954,sainte-genevieve-sur-argence-aveyron-france,3760087843019,d,11,11,None,1029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# we cannot use the nutrition_grade as is. Thus we turn it into a numerical value
def grade_to_number(x):
    if x == "a":
        return 1
    if x == "b":
        return 2
    if x == "c":
        return 3
    if x == "d":
        return 4
    if x == "e":
        return 5
    return 0
global_df['nutrition_grade_numeric'] = global_df['nutrition_grade_fr'].apply(grade_to_number).astype('int')

In [31]:
global_df.to_csv("../data/interim/clean_food_and_cities.csv")

**DISCLAIMER:** Several French towns have the exact same name. Thus, we have a few errors in our mapping. As this situation concerns less than 1% of the towns, it shouldn't impact our analysis.

# Arrondissements

In order to visualize data, we need to make the correspondences between the cities that we have and their respective arrondissement. The arrondissement is the territorial subdivision of the French territory that comes after the Département. There are around 340 arrondissements in France. We thought this subdivision was small enough to show the differences across the French territory but large enough for it to be visible on a map.

In [81]:
global_df = pd.read_csv('../data/interim/clean_food_and_cities.csv')

In [82]:
# The new dataframe:
# https://www.insee.fr/fr/information/2028028
# Table d'appartenance géographique des communes au 1ᵉʳ janvier 2017
city_arr = pd.read_excel('../data/raw/table-appartenance-geo-communes-17.xls', skiprows=[0,1,2,3])
city_arr.drop(0, inplace=True)
city_arr.reset_index(inplace=True, drop=True)

city_arr["city_name"] = city_arr['Libellé géographique'].apply(
    # Remove accents, lowercase, and replace spaces ad apostrophes by '-'
    lambda x: remove_accents(x.lower().replace("'", '-').replace(" ", "-"))
) 

# Drop useless columns
city_arr = city_arr.drop(columns=['Intercommunalité - Métropole', "Nature d'EPCI", "Zone d'emploi 2010", 
                       "Unité urbaine 2010", "Tranche d'unité urbaine 2014", 
                       "Tranche détaillée d'unité urbaine 2014", "Aire urbaine 2010", 
                       "Tranche d'aire urbaine 2014", "Bassin de vie 2012"]
             )
city_arr.rename(columns={'Libellé géographique' : 'City name'}, inplace=True)
city_arr.head()

,Code géographique,City name,Département,Région,Arrondissement,Canton ville,Catégorie commune dans aires urbaines,city_name
0,01001,L'Abergement-Clémenciat,01,84,012,0108,120,l-abergement-clemenciat
1,01002,L'Abergement-de-Varey,01,84,011,0101,112,l-abergement-de-varey
2,01004,Ambérieu-en-Bugey,01,84,011,0101,112,amberieu-en-bugey
3,01005,Ambérieux-en-Dombes,01,84,012,0122,112,amberieux-en-dombes
4,01006,Ambléon,01,84,011,0104,300,ambleon


In [83]:
# to match the geojson file we found, we have to format the code of our arrondissements
city_arr["custom_arrondissement_code"] = city_arr["Département"] + "00" + city_arr['Arrondissement'].astype('str').apply(lambda x: x[-1])
city_arr.head()

,Code géographique,City name,Département,Région,Arrondissement,Canton ville,Catégorie commune dans aires urbaines,city_name,custom_arrondissement_code
0,01001,L'Abergement-Clémenciat,01,84,012,0108,120,l-abergement-clemenciat,01002
1,01002,L'Abergement-de-Varey,01,84,011,0101,112,l-abergement-de-varey,01001
2,01004,Ambérieu-en-Bugey,01,84,011,0101,112,amberieu-en-bugey,01001
3,01005,Ambérieux-en-Dombes,01,84,012,0122,112,amberieux-en-dombes,01002
4,01006,Ambléon,01,84,011,0104,300,ambleon,01001


In [84]:
city_arr.to_csv("../data/interim/city_region_arrondissement.csv")

In [85]:
global_arrond = global_df.merge(city_arr, left_on="city_name", right_on="city_name", how='inner')
global_arrond.head()

,Unnamed: 0,city_index,city_name,food_item_index,city_tag_from_food_item,code,nutrition_grade_fr,nutrition-score-fr_100g,nutrition-score-uk_100g,serving_size,...,Share of taxes (%),nutrition_grade_numeric,Code géographique,City name_y,Département,Région,Arrondissement,Canton ville,Catégorie commune dans aires urbaines,custom_arrondissement_code
0,5,8649,saint-barthelemy-de-bussiere,399027,saint-barthelemy-de-bussiere-dordogne-france,3449350002192,d,16.0,16.0,Une tranche 25 g,...,NaN,4,24381,Saint-Barthélemy-de-Bussière,24,75,242,2413,400,24002
1,6,8649,saint-barthelemy-de-bussiere,399024,saint-barthelemy-de-bussiere-dordogne-france,3449350001782,d,15.0,15.0,NaN,...,NaN,4,24381,Saint-Barthélemy-de-Bussière,24,75,242,2413,400,24002
2,7,8649,saint-barthelemy-de-bussiere,399014,saint-barthelemy-de-bussiere-dordogne-france,3449350000440,d,15.0,15.0,NaN,...,NaN,4,24381,Saint-Barthélemy-de-Bussière,24,75,242,2413,400,24002
3,8,8649,saint-barthelemy-de-bussiere,399025,saint-barthelemy-de-bussiere-dordogne-france,3449350001843,d,16.0,16.0,NaN,...,NaN,4,24381,Saint-Barthélemy-de-Bussière,24,75,242,2413,400,24002
4,9,8649,saint-barthelemy-de-bussiere,399017,saint-barthelemy-de-bussiere-dordogne-france,3449350000754,e,24.0,24.0,NaN,...,NaN,5,24381,Saint-Barthélemy-de-Bussière,24,75,242,2413,400,24002


In [86]:
global_arrond = global_arrond.rename(columns={"Département" : "Department", "Région" : "Region", "City name_y": "City name"})
global_arrond = global_arrond[['food_item_index', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g',
                       'nutrition_grade_numeric', 'serving_size', 'energy_100g', 'fat_100g', 
                       'saturated-fat_100g', 'proteins_100g', 'carbohydrates_100g', 'sugars_100g', 
                       'fiber_100g', 'Median revenue euros', 'Total poverty rate (%)', 'Poverty rate (-30) (%)',
                       'Poverty rate (30-39) (%)', 'Poverty rate (40-49) (%)',
                       'Poverty rate (50-59) (%)', 'Poverty rate (60-74) (%)',
                       'Poverty rate (75+) (%)', 'Poverty rate (house owners) (%)',
                       'Poverty rate (tenants) (%)', 'Share of activity revenue (%)',
                       'Share of retreat pension revenue (%)',
                       'Share of heritage revenue and other (%)',
                       'Share of social benefits revenue (%)', 'Share of taxes (%)', 'City name', 
                       'Department', 'Region', 'custom_arrondissement_code', 'city_name', 'city_tag_from_food_item']]
global_arrond.head()

,food_item_index,nutrition-score-fr_100g,nutrition-score-uk_100g,nutrition_grade_numeric,serving_size,energy_100g,fat_100g,saturated-fat_100g,proteins_100g,carbohydrates_100g,...,Share of retreat pension revenue (%),Share of heritage revenue and other (%),Share of social benefits revenue (%),Share of taxes (%),City name,Department,Region,custom_arrondissement_code,city_name,city_tag_from_food_item
0,399027,16.0,16.0,4,Une tranche 25 g,962.0,12.4,4.6,29.4,0.50,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,24,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
1,399024,15.0,15.0,4,NaN,816.0,9.1,3.3,26.6,1.77,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,24,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
2,399014,15.0,15.0,4,NaN,816.0,9.1,3.3,26.6,1.77,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,24,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
3,399025,16.0,16.0,4,NaN,964.0,12.4,4.6,29.4,0.50,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,24,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
4,399017,24.0,24.0,5,NaN,1502.0,27.2,10.4,26.0,2.50,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,24,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france


In [87]:
global_arrond.to_csv("../data/interim/clean_food_cities_arrond.csv")

# Tax file

In [88]:
departements = (["{:0^3}".format(i) for i in range(1, 20)] +
               ["{:0^3}".format(i) for i in range(21, 96)] +
               # Corsica
               ["2A0", "2B0"] +
               # DOM-TOM
               ["{}".format(i) for i in range(971, 975)] + ["976"])

In [100]:
# Read all the departments-related sheets of the excel files
# The headers are split across rows 19 and 20.
ircom_communes = pd.read_excel("../data/raw/ircom_2017_revenus_2016.xlsx", header=[19, 20],
                               sheet_name=departements
                              )

In [101]:
# We put all the data in one dataframe
ircom_communes_aggregated = None
for _, sheet in ircom_communes.items():
    # format the columns
    sheet.columns = ['_'.join(
                        [column for column in col if not "Unnamed" in column]
                     ).strip().strip('_') 
                     for col in sheet.columns.values
                    ]
    if ircom_communes_aggregated is not None:
        ircom_communes_aggregated = pd.concat([ircom_communes_aggregated, sheet])
    else:
        
        ircom_communes_aggregated = sheet

ircom_communes_aggregated.head()

,Dép.,Commune,Libellé de la commune,Revenu fiscal de référence par tranche (en euros),Nombre de foyers fiscaux,Revenu fiscal de référence des foyers fiscaux,Impôt net (total)*,Nombre de foyers fiscaux imposés,Revenu fiscal de référence des foyers fiscaux imposés,Traitements et salaires_Nombre de foyers concernés,Traitements et salaires_Montant,Retraites et pensions_Nombre de foyers concernés,Retraites et pensions_Montant
NaN,10,1,L'Abergement-Clémenciat,Total,413,12210,692.389,219,8917.02,280,8769.07,155,3802.81
NaN,10,2,L'Abergement-de-Varey,Total,127,3866.62,196.849,67,2776.44,93,2825.99,42,1189.4
NaN,10,4,Ambérieu-en-Bugey,0 à 10 000,1754,7411.82,-15.782,n.c.,n.c.,849,5426.35,467,3989.3
NaN,10,4,Ambérieu-en-Bugey,10 001 à 12 000,497,5476.77,-9.969,n.c.,n.c.,318,4075.28,191,2516.25
NaN,10,4,Ambérieu-en-Bugey,12 001 à 15 000,894,12154.1,-23.811,58,861.66,693,10835.4,227,3352.28


In [102]:
# We keep only the totals per town
ircom_communes_aggregated = ircom_communes_aggregated[
    ircom_communes_aggregated['Revenu fiscal de référence par tranche (en euros)'] == 'Total'
]
# and the relevant columns for our analysis
ircom_communes_aggregated = ircom_communes_aggregated.drop(columns=['Impôt net (total)*', 'Traitements et salaires_Nombre de foyers concernés',
       'Traitements et salaires_Montant',
       'Retraites et pensions_Nombre de foyers concernés',
       'Revenu fiscal de référence des foyers fiscaux imposés',
       'Revenu fiscal de référence par tranche (en euros)',
       'Retraites et pensions_Montant'])
ircom_communes_aggregated.head()

,Dép.,Commune,Libellé de la commune,Nombre de foyers fiscaux,Revenu fiscal de référence des foyers fiscaux,Nombre de foyers fiscaux imposés
NaN,10,1,L'Abergement-Clémenciat,413,12210,219
NaN,10,2,L'Abergement-de-Varey,127,3866.62,67
NaN,10,4,Ambérieu-en-Bugey,8186,194722,3505
NaN,10,5,Ambérieux-en-Dombes,862,25561.9,442
NaN,10,6,Ambléon,59,1755.41,32


In [103]:
# We remove the rows with unknown values
ircom_communes_aggregated = ircom_communes_aggregated[ircom_communes_aggregated['Nombre de foyers fiscaux'] != "n.c."]
ircom_communes_aggregated = ircom_communes_aggregated[
    ircom_communes_aggregated['Revenu fiscal de référence des foyers fiscaux'] != "n.c."
]
ircom_communes_aggregated = ircom_communes_aggregated[
    ircom_communes_aggregated['Nombre de foyers fiscaux imposés'] != "n.c."
]

In [104]:
# give them the right type
ircom_communes_aggregated['Nombre de foyers fiscaux'] = ircom_communes_aggregated[
    'Nombre de foyers fiscaux'].astype('int')
ircom_communes_aggregated['Revenu fiscal de référence des foyers fiscaux'] = ircom_communes_aggregated[
    'Revenu fiscal de référence des foyers fiscaux'].astype('float')
ircom_communes_aggregated['Nombre de foyers fiscaux imposés'] = ircom_communes_aggregated[
    'Nombre de foyers fiscaux imposés'].astype('int')

In [105]:
# We add a column for the mean fiscal revenue per household
ircom_communes_aggregated['Mean reference fiscal income (in k€)'] = ircom_communes_aggregated[
    "Revenu fiscal de référence des foyers fiscaux"
] / ircom_communes_aggregated['Nombre de foyers fiscaux']

In [106]:
# we format the department names and the city names to match our previous patterns
ircom_communes_aggregated['Dép.'] = ["{:0>3}".format(i) for i in ircom_communes_aggregated['Dép.']]
ircom_communes_aggregated['city_tag'] = ircom_communes_aggregated['Libellé de la commune'].apply(
    lambda x: remove_accents(x.lower().replace("'", "-").replace(" ", "-"))
)
ircom_communes_aggregated['Dép.'] = ircom_communes_aggregated['Dép.'].astype('str')
ircom_communes_aggregated.head()

,Dép.,Commune,Libellé de la commune,Nombre de foyers fiscaux,Revenu fiscal de référence des foyers fiscaux,Nombre de foyers fiscaux imposés,Mean reference fiscal income (in k€),city_tag
NaN,010,1,L'Abergement-Clémenciat,413,12209.992,219,29.564145,l-abergement-clemenciat
NaN,010,2,L'Abergement-de-Varey,127,3866.621,67,30.445835,l-abergement-de-varey
NaN,010,4,Ambérieu-en-Bugey,8186,194722.047,3505,23.787203,amberieu-en-bugey
NaN,010,5,Ambérieux-en-Dombes,862,25561.917,442,29.654196,amberieux-en-dombes
NaN,010,6,Ambléon,59,1755.412,32,29.752746,ambleon


In [107]:
# we retrieve the last dataframe
global_arrond = pd.read_csv("../data/interim/clean_food_cities_arrond.csv")
global_arrond['Department'] = ["{:0^3}".format(d) for d in global_arrond['Department']]
global_arrond.head()

,Unnamed: 0,food_item_index,nutrition-score-fr_100g,nutrition-score-uk_100g,nutrition_grade_numeric,serving_size,energy_100g,fat_100g,saturated-fat_100g,proteins_100g,...,Share of retreat pension revenue (%),Share of heritage revenue and other (%),Share of social benefits revenue (%),Share of taxes (%),City name,Department,Region,custom_arrondissement_code,city_name,city_tag_from_food_item
0,0,399027,16.0,16.0,4,Une tranche 25 g,962.0,12.4,4.6,29.4,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,240,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
1,1,399024,15.0,15.0,4,NaN,816.0,9.1,3.3,26.6,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,240,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
2,2,399014,15.0,15.0,4,NaN,816.0,9.1,3.3,26.6,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,240,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
3,3,399025,16.0,16.0,4,NaN,964.0,12.4,4.6,29.4,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,240,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france
4,4,399017,24.0,24.0,5,NaN,1502.0,27.2,10.4,26.0,...,NaN,NaN,NaN,NaN,Saint-Barthélemy-de-Bussière,240,75,24002,saint-barthelemy-de-bussiere,saint-barthelemy-de-bussiere-dordogne-france


In [108]:
# we merge the dataframe with all the features with the one containing the mean fiscal income
global_arrond = pd.merge(ircom_communes_aggregated, global_arrond, 
                         left_on=['city_tag', 'Dép.'], right_on=['city_name', 'Department'])


In [109]:
# we drop unneeded columns, and roughly translate the French labels
global_arrond.drop(columns=["Dép.", "Commune", "Libellé de la commune", "Revenu fiscal de référence des foyers fiscaux"],
                   inplace=True)
global_arrond.rename(columns={"Nombre de foyers fiscaux": "Fiscal household number", 
                              "Nombre de foyers fiscaux imposés": "Taxed households number",
                             })
# and we save our dataframe for later
global_arrond.to_csv("../data/processed/clean_food_cities_arrond_with_tax.csv")